# GWP Submission 2: Pricing Options under Binomial Models

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csc_matrix
from scipy.stats import bernoulli
from scipy.optimize import fsolve
from scipy.stats import norm
import pandas as pd
from matplotlib import colors
import seaborn as sns
from pandas import DataFrame
from math import exp,sqrt
%matplotlib inline

Write code to price a European Call option:


    Group Number = 25
    𝑢 = (1.10 + Group Number/100)


In [2]:
# Our initial stock price
S0 = 95
# Time to expiration, the we rescale it to time per layer step
t = 1
# Strike price and risk-free rate
K = 105
r = 0
# Calculate the size of an up move, down move, and probability
Gn = 25 # Group Number
u = (1.10 + Gn/100)
d = 1/u
p = (np.exp(r * t) - d) / (u - d)

Pricing a European Call Option:

In [3]:
def binom_tree_call(N, T, S0, r, K, array_out=False):
    # Init
    dt = T/N
    u = (1.10 + 25/100)
    d = 1/u
    p = (np.exp(r*dt)-d)/(u-d)
    
    #Price Tree
    price_tree = np.zeros([N+1, N+1])
    
    for i in range(N+1):
        for j in range(i+1):
            price_tree[j, i] = S0*(d**j)*(u**(i-j))
            
    # Option Value
    option = np.zeros([N+1, N+1])
    option[:, N] = np.maximum(np.zeros(N+1), price_tree[:, N]-K)
    
    # Calculate option price at t=0
    for i in np.arange(N-1, -1, -1):
        for j in np.arange(0, i+1):
            option[j, i] = np.exp(-r*dt)*(p*option[j, i+1]+(1-p)*option[j+1, i+1])
            
    #Return
    if array_out:
        return [option[0, 0], price_tree, option]
    else:
        return option[0, 0]

Value of the derivative, 𝐻(𝜔), for each of the paths:

In [4]:
binom_tree_call(1, 1, 95, 0, 105, True)

[9.893617021276595,
 array([[ 95.        , 128.25      ],
        [  0.        ,  70.37037037]]),
 array([[ 9.89361702, 23.25      ],
        [ 0.        ,  0.        ]])]

Pricing a European Put Option, N=2:

In [5]:
def binom_tree_call_put(N, T, SO, r, K, call=True, array_out=False):
    # Init
    dt = T/N
    u = (1.10 + 25/100)
    d = 1/u
    p = (np.exp(r*dt)-d)/(u-d)
    
    #Price Tree
    price_tree = np.zeros([N+1, N+1])
    
    for i in range(N+1):
        for j in range(i+1):
            price_tree[j, i] = S0*(d**j)*(u**(i-j))
            
    # Option Value
    option = np.zeros([N+1, N+1])
    if call:
        option[:, N] = np.maximum(np.zeros(N+1), price_tree[:, N]-K)
    else:
        option[:, N] = np.maximum(np.zeros(N+1), K-price_tree[:, N])
    
    # Calculate option price at t=0
    for i in np.arange(N-1, -1, -1):
        for j in np.arange(0, i+1):
            option[j, i] = np.exp(-r*dt)*(p*option[j, i+1]+(1-p)*option[j+1, i+1])
            
    #Return
    if array_out:
        return [option[0, 0], price_tree, option]
    else:
        return option[0, 0]

Price of the option for N=2:

In [6]:
binom_tree_call_put(N=2, T=1, SO=95, r=0, K=105, call=False, array_out=True)

[22.338162064282486,
 array([[ 95.        , 128.25      , 173.1375    ],
        [  0.        ,  70.37037037,  95.        ],
        [  0.        ,   0.        ,  52.12620027]]),
 array([[22.33816206,  5.74468085,  0.        ],
        [ 0.        , 34.62962963, 10.        ],
        [ 0.        ,  0.        , 52.87379973]])]

Put Call Parity:
* Price of the European put option for N=1:

In [7]:
binom_tree_call_put(N=1, T=1, SO=95, r=0, K=105, call=False, array_out=True)

[19.8936170212766,
 array([[ 95.        , 128.25      ],
        [  0.        ,  70.37037037]]),
 array([[19.89361702,  0.        ],
        [ 0.        , 34.62962963]])]